In [ ]:
import json
import os

import datapane as dp
import matplotlib
import matplotlib.pyplot as plt
import skimage
from skimage import data, img_as_ubyte
from skimage.color import gray2rgb, rgb2gray, yuv2rgb
from skimage.filters import roberts
from skimage.io import imread, imsave
from skimage.morphology import skeletonize

## Globals

In [ ]:
img = data.astronaut()
operation_history = []

## Custom operations

In [ ]:
def tint_red(img):
    multiplier = [1, 0, 0]
    img = img * multiplier
    return img


def tint_yellow(img):
    multiplier = [1, 1, 0]
    img = img * multiplier
    return img

## Utils

In [ ]:
def convert_bytes(num):
    """
    this function will convert bytes to MB.... GB... etc
    """
    for x in ["bytes", "KB", "MB", "GB", "TB"]:
        if num < 1024.0:
            return "%3.1f %s" % (num, x)
        num /= 1024.0

## Image operation result row (Datapane blocks)

In [ ]:
def operation_result(last_operation):
    global img
    global operation_history

    imsave("img.png", img)

    with open("operation.json", "w") as f:
        json.dump(operation_history, f)

    return dp.Group(
        dp.Media(file="img.png"),
        dp.Group(
            dp.Group(
                dp.Text(
                    f"""## Metadata
- **Last operation**: {last_operation}
- **Axes**: {img.ndim}
- **Dimensions**: {img.shape}
- **Size**: {img.size}
- **Type**: {img.dtype}
- **File size**: {convert_bytes(os.path.getsize('img.png'))}"""
                ),
            ),
            dp.Group(
                dp.Text("## Export"),
                dp.Group(
                    dp.Attachment(file="img.png"),
                    dp.Attachment(file="operation.json"),
                    columns=2,
                ),
            ),
        ),
        columns=2,
    )

## Function to operate on an image, returns operation result

In [ ]:
def image_operation(params):
    global img
    global operation_history
    
    operation_history.append(params["operation"])
    img = globals()[params["operation"]](img)

    return dp.View(operation_result(params["operation"]))

## Function to load a new image, returns operation result

In [ ]:
def load_image(params):
    global img
    global operation_history

    operation_history = []
    caller = getattr(data, params["image"])
    img = caller()

    return dp.View(operation_result(f"load image ({params['image']})"))

## Datapane Controls for image manipulation

In [ ]:
operation_controls = dp.Controls(
    dp.Choice(
        "operation",
        label="Select an operation",
        default="rgb2gray",
        options=[
            "rgb2gray",
            "gray2rgb",
            "tint_red",
            "tint_yellow",
            "skeletonize",
            "roberts",
        ],
    ),
)


image_controls = dp.Controls(
    dp.Choice(
        "image",
        label="Load image & restart",
        default="astronaut",
        options=[
            "astronaut",
            "binary_blobs",
            "brick",
            "colorwheel",
            "camera",
            "cat",
            "checkerboard",
            "clock",
            "coffee",
            "coins",
            "eagle",
            "grass",
            "gravel",
            "horse",
            "logo",
            "page",
            "text",
            "rocket",
        ],
    ),
)

## Construct Datapane App and serve

In [ ]:
v = dp.View(
    dp.Group(
        dp.Function(
            image_operation,
            target="image_history",
            controls=operation_controls,
            swap=dp.Swap.PREPEND,
        ),
        dp.Function(
            load_image,
            target="image_history",
            controls=image_controls,
            swap=dp.Swap.INNER,
        ),
        columns=2,
    ),
    dp.Group(operation_result("load image"), name="image_history"),
)


dp.serve(v)